In [1]:

import nltk
import numpy as np
from textblob import TextBlob
import pandas as pd 
import glob
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold # import KFold
from sklearn.model_selection import KFold, StratifiedKFold
#Load libraries needed for classification 
from sklearn import naive_bayes
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.naive_bayes import BernoulliNB, MultinomialNB,GaussianNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
#from sklearn.naive_bayes import ComplementNB
#from bayes.classifiers import ComplementNB
#from naiveBayesClassifier.classifier import Classifier
from sklearn import svm, datasets
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize, pos_tag
from nltk.util import ngrams
from nltk.corpus import wordnet as omw # open multilingual wordnet
from nltk.stem.isri import ISRIStemmer

In [2]:
# A.read the course names training data & apply preprocessing into a data frame
# A.1 upload  Train data file names
path =r'D:\MahmoudOut\Final Project\CoursesAnalysis-master'
ds1_train=pd.read_excel(path+"\TrainData.xlsx")

In [3]:
#get Arabic sentences only
import re

def ArabicOnly(text):
    #print (text)
    result = re.sub(r'[^ء-ي]',' ',str(text)) #arabic
    if len(result.strip()) >0:
        return 'T'
    else:
        return 'F'
ds1_train['ArabicOnly'] =ds1_train['valueName'].apply(lambda x : ArabicOnly(x))


In [4]:
ds1_train = ds1_train[ds1_train.ArabicOnly =='T']
len(ds1_train)

79

In [5]:
# Arabic diacritics
arabic_punct = ''' ` ÷ × ؛ < > _ ( ) * & ^ % ] [ ـ ، / : " ؟ . , ' { } ~ ¦ + | !  ”  …  “ –   ـ  '''
arabic_diacritics = '''  َ     ُ       ِ      ّ       ً      ٌ       ٍ      ْ     '''

arabic_punct = arabic_punct.split()

arabic_diacritics = arabic_diacritics.split()

# Arabic punctuations and dicritis 
punctuations = set( arabic_punct  + arabic_diacritics )

# Arabic stopwords. This list are obtained from http://www.ranks.nl/stopwords/arabic 
asw = open(path+'/ArabicStopwords.txt', encoding="utf8").read()

arabicStopWords =  asw.split() #+ aswUinicode.split()

asw2 = ''

# Arabic  stopwords
all_stopwords = set(arabicStopWords)


In [6]:
all_stopwords

{'آخر',
 'أ',
 'أكثر',
 'ألا',
 'أما',
 'أن',
 'أنا',
 'أنت',
 'أنفسكم',
 'أنفسنا',
 'أنفسهم',
 'أنه',
 'أنها',
 'أو',
 'أى',
 'أي',
 'أيضا',
 'إذا',
 'إضافي',
 'إلا',
 'إلى',
 'إلي',
 'إما',
 'إن',
 'إنه',
 'إنها',
 'ا',
 'اذا',
 'اكثر',
 'الآن',
 'الا',
 'الان',
 'التى',
 'التي',
 'الذى',
 'الذي',
 'الذين',
 'الى',
 'الي',
 'اما',
 'ان',
 'انت',
 'انه',
 'انها',
 'او',
 'اى',
 'اي',
 'ايضا',
 'ب',
 'بأن',
 'بان',
 'بسبب',
 'بشكل',
 'بعد',
 'بعض',
 'بعيدا',
 'بن',
 'به',
 'بها',
 'بين',
 'بينما',
 "تحت'",
 'تكون',
 'تلك',
 'تم',
 'ثانية',
 'ثم',
 'جدا',
 'جميع',
 'حاليا',
 'حتى',
 'حول',
 'حيث',
 'حين',
 'خارج',
 'خلال',
 'دون',
 'ديك',
 'ذلك',
 'سوف',
 'صباحا',
 'ضد',
 'ضمن',
 'على',
 'عليه',
 'عليها',
 'عن',
 'عند',
 'عندما',
 'غير',
 'ف',
 'فأن',
 'فان',
 'فعل',
 'فقط',
 'فما',
 'فهو',
 'فهى',
 'فهي',
 'فوق',
 'فى',
 'في',
 'فيه',
 'فيها',
 'قبل',
 'قد',
 'قليل',
 'كان',
 'كانت',
 'كل',
 'كلا',
 'كما',
 'كيف',
 'ل',
 'لا',
 'لدى',
 'لديه',
 'لقاء',
 'لك',
 'لكن',
 'لم',
 'لماذا',
 

In [7]:
# remove punctcutions
def remove_punct(word):
    for c in word: return ''.join(ch for ch in word if not ch in punctuations) # remove punctuation
###################################################################################

In [8]:
# remove arabic diacritics
def remove_diacritics(text):
    arstemmer = ISRIStemmer()
    result = arstemmer.norm(text, num=1) #  remove diacritics which representing Arabic short vowels
    return result
###################################################################################

In [9]:
# takes a word list and returns the root for each Arabic words
def getRootAr(word_list):
    result = []
    arstemmer = ISRIStemmer()
    for word in word_list: result.append(arstemmer.stem(word))
    return result#' '.join(result)

In [30]:
arstemmer = ISRIStemmer()
print(arstemmer.pre32('الوقت'))
print(arstemmer.suf32('بنائون'))
print(arstemmer.waw('وورد'))
print(arstemmer.norm('أحمد', num=2))


وقت
بنائ
ورد
احمد


In [28]:
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text

In [33]:
print(normalize_arabic('ؤ'))

ء


In [32]:
# Arabic light stemming for Arabic text
# takes a word list and perform light stemming for each Arabic words
def lightStemAr(word_list):
    result = []
    arstemmer = ISRIStemmer()
    for word in word_list:
        word = arstemmer.norm(word, num=1)  #  remove diacritics which representing Arabic short vowels  
        if not word in arstemmer.stop_words:   # exclude stop words from being processed
            word = arstemmer.pre32(word)        # remove length three and length two prefixes in this order
            word = arstemmer.suf32(word)        # remove length three and length two suffixes in this order
            word = arstemmer.waw(word)          # remove connective ‘و’ if it precedes a word beginning with ‘و’
            word = arstemmer.norm(word, num=2)       # normalize initial hamza to bare alif
            word = normalize_arabic(word)
        result.append(word)
    return result#' '.join(result)

In [11]:
#remove longation
def removeLongation(word_list):
    result = []
    for word in word_list:
        #remove longation
        p_longation = re.compile(r'(.)\1+')
        subst = r"\1"
        word = re.sub(p_longation, subst, word) 
        result.append(word)
    return result

In [12]:
# takes a string of text and returns the word list (tonkized words)
# processing includes: removing diacritics and punctcutions, removing stopwords, and tokenizing
# Stemmning
# Get Root
def process_text(text, removePunct=True, removeSW=True, removeNum=True, Stem=True, Root=True):
    text = remove_diacritics(text)# remove arabic diacritics
    word_list = nltk.tokenize.wordpunct_tokenize(text.lower())
    word_list = removeLongation(word_list) #remove longation
    if removePunct:
        word_list = [ w for w in word_list if not w in punctuations ]
        word_list = [ remove_punct(w) for w in word_list ]
    if removeSW: word_list = [ w for w in word_list if not w in all_stopwords ]
    if removeNum: word_list = [ w for w in word_list if not w.isdigit() ]
    if Stem: word_list = lightStemAr(word_list) #strimming
    if Root: word_list = getRootAr(word_list) #get Root
        
       
    return ' '.join(word_list)
###################################################################################

In [ ]:
ds1_train['preprocessingWithStem'] =ds1_train['valueName'].apply(lambda x : process_text(x,True,True,False,True,False))

In [ ]:
ds1_train['preprocessingWithRoot'] =ds1_train['valueName'].apply(lambda x : process_text(x,True,True,False,True,True))

In [ ]:
ds1_train

In [ ]:
#word Embeddings
# -*- coding: utf8 -*-
import gensim
# load the model
model = gensim.models.Word2Vec.load(path+'\Twt-CBOW\\Twt-CBOW')


In [ ]:
#pip install hunspell
#pip install aspell-python-py3

In [38]:
#spell Checker
from spellchecker import SpellChecker

spell = SpellChecker(language='ar')  # Arabic dictionary
print(spell['السلام'])

ValueError: The provided dictionary language (ar) does not exist!